<a href="https://colab.research.google.com/github/siddharth-kale/DNA3/blob/main/DNA3_Emirates_Flight_Status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install plotly.express

In [2]:
import pandas as pd
import numpy as np
import seaborn as sea
import plotly_express as px
from google.colab import drive
import glob

In [3]:
pd.options.display.max_rows=999

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
emirates_df = pd.DataFrame()

files = glob.glob('/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/*.csv')

for f in files:
  temp_df = pd.read_csv(f).drop(columns=['Unnamed: 0'])
  print(f)
  emirates_df = emirates_df.append(temp_df)

emirates_df 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.



/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2020-11-18.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2020-12-06.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (3,7) have mixed types.Specify dtype option on import or set low_memory=False.



/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-01-03.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-02-15.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-03-07.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-04-09.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-05-10.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-06-12.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-07-27.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-08-20.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-09-06.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-10-03.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (3,7,8) have mixed types.Specify dtype option on import or set low_memory=False.



/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2020-10-15-005.csv
/content/gdrive/MyDrive/Projects/DNA3/Data/emirates/2021-11-02.csv


,alert,altitude,callsign,geoaltitude,groundspeed,hour,icao24,last_position,latitude,longitude,onground,spi,squawk,timestamp,track,vertical_rate
0,False,NaN,NaN,NaN,546.0,2020-11-17 00:00:00+00:00,89611c,NaN,NaN,NaN,False,False,NaN,2020-11-17 00:00:17+00:00,42.478240,1088.0
1,False,29100.0,NaN,NaN,547.0,2020-11-17 00:00:00+00:00,89611c,2020-11-17 00:00:17.952000+00:00,26.815020,55.962031,False,False,NaN,2020-11-17 00:00:18+00:00,42.703470,1088.0
2,False,29125.0,NaN,30500.0,548.0,2020-11-17 00:00:00+00:00,89611c,2020-11-17 00:00:18.494999808+00:00,26.816044,55.963035,False,False,NaN,2020-11-17 00:00:19+00:00,42.780344,1088.0
3,False,29150.0,UAE322,30500.0,548.0,2020-11-17 00:00:00+00:00,89611c,2020-11-17 00:00:19.586999808+00:00,26.818085,55.965133,False,False,NaN,2020-11-17 00:00:20+00:00,42.857029,1088.0
4,False,29150.0,UAE322,30500.0,548.0,2020-11-17 00:00:00+00:00,89611c,2020-11-17 00:00:20.569999872+00:00,26.819908,55.967049,False,False,NaN,2020-11-17 00:00:21+00:00,42.857029,1088.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2793496,False,41000.0,UAE2624,42375.0,516.0,2021-11-01 23:00:00+00:00,8964b2,2021-11-01 23:59:54.898000128+00:00,29.575102,32.379886,False,False,3345.0,2021-11-01 23:59:55+00:00,84.554408,0.0
2793497,False,41000.0,UAE2624,42375.0,516.0,2021-11-01 23:00:00+00:00,8964b2,2021-11-01 23:59:55.931000064+00:00,29.575333,32.382519,False,False,3345.0,2021-11-01 23:59:56+00:00,84.554408,0.0
2793498,False,41000.0,UAE2624,42375.0,516.0,2021-11-01 23:00:00+00:00,8964b2,2021-11-01 23:59:56.891000064+00:00,29.575568,32.385164,False,False,3345.0,2021-11-01 23:59:57+00:00,84.554408,0.0
2793499,False,41000.0,UAE2624,42375.0,517.0,2021-11-01 23:00:00+00:00,8964b2,2021-11-01 23:59:57.737999872+00:00,29.575745,32.387326,False,False,3345.0,2021-11-01 23:59:58+00:00,84.564918,0.0


In [6]:
emirates_df['date'] = emirates_df['timestamp'].str[:10]

In [7]:
emirates_df['icao24'] = emirates_df['icao24'].astype(dtype='string')
emirates_df['date'] = emirates_df['date'].astype(dtype='string')

In [8]:
emirates_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62091583 entries, 0 to 2793500
Data columns (total 17 columns):
 #   Column         Dtype  
---  ------         -----  
 0   alert          bool   
 1   altitude       float64
 2   callsign       object 
 3   geoaltitude    float64
 4   groundspeed    float64
 5   hour           object 
 6   icao24         string 
 7   last_position  object 
 8   latitude       float64
 9   longitude      float64
 10  onground       bool   
 11  spi            bool   
 12  squawk         float64
 13  timestamp      object 
 14  track          float64
 15  vertical_rate  float64
 16  date           string 
dtypes: bool(3), float64(8), object(4), string(2)
memory usage: 7.1+ GB


In [10]:
fleet_df = emirates_df.groupby(by=['date','icao24']).agg(
    {
        'groundspeed':['mean']
    }
).reset_index()

fleet_df.columns=['date','ICAO','groundspeed']

fleet_df

,date,ICAO,groundspeed
0,2019-12-07,8960.0,485.873449
1,2019-12-07,8960000.0,519.254298
2,2019-12-07,8960000000000.0,430.030558
3,2019-12-07,8960c9,402.996168
4,2019-12-07,8960cc,292.094590
...,...,...,...
3386,2021-11-01,896480,440.239340
3387,2021-11-01,896481,440.694223
3388,2021-11-01,89649f,446.746140
3389,2021-11-01,8964b1,440.378870


In [11]:
final_df = fleet_df[fleet_df['groundspeed'].notna()].groupby(by=['date']).agg({
    
    'groundspeed':['mean'],
    'ICAO':['count']

}
).reset_index()
final_df.columns=['Date','Groundspeed','Aircraft']
final_df['Date'] = final_df['Date'].str[:7]
final_df['Proportion'] = round(final_df['Aircraft']/final_df['Aircraft'].max(),2)
final_df

,Date,Groundspeed,Aircraft,Proportion
0,2019-12,393.097975,235,0.99
1,2020-01,392.129435,238,1.00
2,2020-02,413.133990,223,0.94
3,2020-03,408.716209,219,0.92
4,2020-04,402.211727,39,0.16
5,2020-05,411.487105,58,0.24
6,2020-06,400.715374,81,0.34
7,2020-07,401.048072,98,0.41
8,2020-08,401.644393,108,0.45
9,2020-09,411.646577,110,0.46


In [12]:
fig = px.bar(final_df, y='Aircraft', x='Date',text='Aircraft')
fig.update_traces(textposition='outside',marker_color='rgb(10,10,105)',)
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
final_df.to_csv('/content/gdrive/MyDrive/AlternativeData/DNA3/flights/Plots/emiratesairborne.csv')

In [ ]:
#emirates_df.to_csv('/content/gdrive/MyDrive/AlternativeData/DNA3/flights/totalemirates.csv')